In [ ]:
# we do not need to generator 32 channels together. One way is to generate them 

import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, inputsize, hiddensize, outputsize):
        super(Generator, self).__init__()
        self.inputsize = inputsize # 8, 16, 24
        self.outputsize = outputsize
        self.hiddensize = hiddensize
        self.section1 = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(self.inputsize, hiddensize * 8, 3, 1, padding=1, bias=False, dilation=1),
            nn.BatchNorm2d(hiddensize * 8),
            nn.ReLU(True)
        )
        self.section2 = nn.Sequential(
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(hiddensize * 8, hiddensize * 4, 3, 1, padding=1 , bias=False, dilation=1),
            nn.BatchNorm2d(hiddensize * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(hiddensize * 4, hiddensize * 2, 3, 1, padding=1, bias=False, dilation=1),
            nn.BatchNorm2d(hiddensize * 2),
            nn.ReLU(True),
            # # state size. (ngf*2) x 16 x 16
            # nn.ConvTranspose2d(hiddensize * 2, hiddensize, 4, 1, 0, bias=False),
            # nn.BatchNorm2d(hiddensize),
            # nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d(hiddensize * 2, self.outputsize, 3, 1, padding=1, bias=False, dilation=1),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        output = self.section1(input)
        output = self.section2(output)
        return output

In [6]:
# use the client and server for mobilenetv2
import torch
from Models.mobilenetv2 import mobilenetv2_splitter, MobileNetV2
client, server = mobilenetv2_splitter(num_classes = 10, 
                                      weight_root = '/home/tonypeng/Workspace1/adaptfilter/Adaptfilter/Weights/cifar-10', 
                                      device = 'cuda:0', 
                                      partition = -1)
model = MobileNetV2(num_classes = 10)
model.load_state_dict(torch.load('/home/tonypeng/Workspace1/adaptfilter/Adaptfilter/Weights/cifar-10/pretrained/mobilenetv2.pth'))
# get the dataloader 
from Dataloaders import dataloader_cifar10
train, test, classes = dataloader_cifar10.Dataloader_cifar10()

client = client.to('cuda:0')
server = server.to('cuda:0')
model = model.to('cuda:0')
client.eval()
server.eval()
model.eval()

with torch.no_grad():
    correct = 0
    correct2 = 0
    for i, data in enumerate(test, 0):
        inputs, labels = data
        inputs, labels = inputs.to('cuda:0'), labels.to('cuda:0')
        outputs = client(inputs)
        outputs = server(outputs)
        outputs2 = model(inputs)
        outputs = torch.argmax(outputs, dim = 1)
        outputs2 = torch.argmax(outputs2, dim = 1)
        correct_rate = (outputs == labels).sum().item()
        correct_rate2 = (outputs2 == labels).sum().item()
        correct += correct_rate
        correct2 += correct_rate2
    print('Total Accuracy ', correct/10000)


Files already downloaded and verified
Files already downloaded and verified
Total Accuracy  0.8284


In [1]:
# add a squeeze layer to the model
import torch
import torch.nn as nn
class depressor(nn.Module):
    def __init__(self, inputsize, outputsize):
        # ex input 16, output 4
        super().__init__()
        self.inputsize = inputsize
        self.outputsize = outputsize
        self.section1 = nn.Sequential(
            nn.Conv2d(self.inputsize, self.outputsize, 3, 1, padding = 1),
            nn.BatchNorm2d(self.outputsize),
        )
    def forward(self, input):
        output = self.section1(input)
        return output
# https://github.com/Lornatang/CGAN-PyTorch/blob/master/cgan_pytorch/models/generator.py
class Generator(nn.Module):
    def __init__(self, image_size: int = 28, channels: int = 1, num_channels: int = 32):
        # ex input 4, output 1
        super().__init__()
        self.image_size = image_size
        self.channels = channels

        self.label_embedding = nn.Embedding(num_channels, num_channels)
        self.input_size = self.image_size[0] * self.image_size[1]
        self.input_size += num_channels
        self.output_size = self.image_size[0] * self.image_size[1]
        
        self.pre = nn.Sequential(
            nn.Conv2d(self.input_size, self.input_size//2, 3, 1, padding=1),
            nn.BatchNorm2d(self.input_size//2),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            
            nn.Conv2d(self.input_size//2, 1, 3, 1, padding=1),
            nn.BatchNorm2d(1),
            nn.LeakyReLU(negative_slope=0.2, inplace=True)
        )
        self.main = nn.Sequential(
            nn.Linear(self.input_size, 2*self.input_size),
            nn.BatchNorm1d(2*self.input_size),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),

            nn.Linear(2*self.input_size, 4*self.input_size),
            nn.BatchNorm1d(4*self.input_size),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),

            nn.Linear(4*self.input_size, self.input_size),
            nn.Tanh()
        )

    def forward(self, inputs: torch.Tensor, labels: list = None) -> torch.Tensor:

        conditional_inputs = torch.cat([inputs, self.label_embedding(labels)], dim=-1)
        out = self.main(conditional_inputs)
        out = out.reshape(self.image_size, self.image_size)

In [3]:
# import client and server
from Models.mobilenetv2 import mobilenetv2_splitter, MobileNetV2
client, server = mobilenetv2_splitter(num_classes = 10, weight_root='/home/tonypeng/Workspace1/adaptfilter/Adaptfilter/Weights/cifar-10', device='cuda:0', partition=-1)
from Dataloaders import dataloader_cifar10
train, test, classes = dataloader_cifar10.Dataloader_cifar10()
client = client.to('cuda:0')
server = server.to('cuda:0')
client.eval()
server.eval()

depress = depressor(16, 4)
depress = depress.to('cuda:0')
depress.train()

generator = Generator(image_size=[112,112])
generator = generator.to('cuda:0')
generator.train()

import torch.optim as optim
criterion = nn.L1Loss()
d_optimizer = optim.SGD(depress.parameters(), lr=0.001, momentum=0.9)
g_optimizer = optim.SGD(generator.parameters(), lr=0.001, momentum=0.9)

print('?block')
import numpy as np
def get_zero_rank(input):
    # input is torch
    inputsize = torch.prod(input.size(2) * input.size(3))# 4*4
    zero_rate = torch.zeros(input.shape[:2]) # b,c
    zero_rank = torch.zeros(input.shape[:2]) # b,c
    for i, c in enumerate(input):
        for j in range(input[i].shape[0]): # c
            zeros = torch.where(input[i,j,:,:]==0, 1, 0)
            zero_rate[i,j] = torch.sum(zeros)/inputsize
            if zero_rate[i,j] > 0.5:
                zero_rate[i,j] = 1
        # sort the zeros rate
        zero_rank[i] = torch.argsort(zero_rate) # descend 
    return zero_rank

test = torch.zeros(2,3,4,4)
test[0,0,:,:] = 1
test[0,1,:,0:3] = 1
print(get_zero_rank(test))
# for epoch in range(10):
#     running_loss = 0.0
#     for i, data in enumerate(train, 0):
#         inputs, labels = data
#         inputs, labels = inputs.to('cuda:0'), labels.to('cuda:0')
#         outputs = client(inputs)

        # get zeros
        

Files already downloaded and verified
Files already downloaded and verified
?block


TypeError: prod(): argument 'input' (position 1) must be Tensor, not int

: 

In [14]:
import torch
def get_zero_rank(input):
    # input is torch
    inputsize = input.size(2) * input.size(3)# 4*4
    zero_rate = torch.zeros(input.shape[:2]) # b,c
    zero_rank = torch.zeros(input.shape[:2]) # b,c
    for i, c in enumerate(input):
        for j in range(input[i].shape[0]): # c
            zeros = torch.where(input[i,j,:,:]==0, 1, 0)
            zero_rate[i,j] = torch.sum(zeros)/inputsize
            if zero_rate[i,j] > 0.5:
                zero_rate[i,j] = 1
        # sort the zeros rate
    zero_rank = torch.argsort(zero_rate) # descend 
        
    return zero_rank
test = torch.zeros(2,3,4,4)
test[0,2,:,:] = 1
test[0,0,:,0:3] = 1
print(get_zero_rank(test))


tensor([[2, 0, 1],
        [0, 1, 2]])


In [21]:
a = torch.tensor([[1,2,3],[4,5,6]])
b = torch.tensor([1, 0])
a[b,:]

tensor([[4, 5, 6],
        [1, 2, 3]])